# CIFAR-10

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [1]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model , Sequential
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [2]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.6.0
Nom du GPU : /device:GPU:0


## Importation du dataset

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

In [4]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [5]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [7]:
LOG_DIR = os.path.join("logs") 
MODEL_DIR = os.path.join("models") 

## Fixer les seeds

In [8]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# ConvNet

## Learning rate et Momentum 
### Hyperparamètre : 
#### NUM_LAYERS = [5,10,15]
#### OPTIMIZER = SGD
#### BATCH_SIZE = 512
#### EPOCHS = 200 
#### learning_rates = [0.5, 0.1, 0.05,0 .01, 0.005, 0.001]
#### momentums = [0.25, 0.5, 0.9]

In [9]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True

In [10]:
learning_rates = [0.01, 0.005, 0.001]
momentums = [0.25, 0.5, 0.9]


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)
                       

Epoch 1/300
98/98 [==============================] - 21s 25ms/step - loss: 2.2861 - categorical_accuracy: 0.1534 - val_loss: 2.2677 - val_categorical_accuracy: 0.1850
Epoch 2/300
98/98 [==============================] - 2s 18ms/step - loss: 2.2456 - categorical_accuracy: 0.2266 - val_loss: 2.2181 - val_categorical_accuracy: 0.2524
Epoch 3/300
98/98 [==============================] - 2s 17ms/step - loss: 2.1746 - categorical_accuracy: 0.2686 - val_loss: 2.1193 - val_categorical_accuracy: 0.3050
Epoch 4/300
98/98 [==============================] - 2s 18ms/step - loss: 2.0558 - categorical_accuracy: 0.2937 - val_loss: 1.9971 - val_categorical_accuracy: 0.3115
Epoch 5/300
98/98 [==============================] - 2s 18ms/step - loss: 1.9674 - categorical_accuracy: 0.3084 - val_loss: 1.9335 - val_categorical_accuracy: 0.3230
Epoch 6/300
98/98 [==============================] - 2s 18ms/step - loss: 1.9118 - categorical_accuracy: 0.3238 - val_loss: 1.8799 - val_categorical_accuracy: 0.3398
Epo

98/98 [==============================] - 2s 18ms/step - loss: 0.7608 - categorical_accuracy: 0.7388 - val_loss: 0.9014 - val_categorical_accuracy: 0.6839
Epoch 100/300
98/98 [==============================] - 2s 18ms/step - loss: 0.7535 - categorical_accuracy: 0.7405 - val_loss: 0.8901 - val_categorical_accuracy: 0.6856
Epoch 101/300
98/98 [==============================] - 2s 18ms/step - loss: 0.7492 - categorical_accuracy: 0.7426 - val_loss: 0.8676 - val_categorical_accuracy: 0.6975
Epoch 102/300
98/98 [==============================] - 2s 18ms/step - loss: 0.7450 - categorical_accuracy: 0.7432 - val_loss: 0.8775 - val_categorical_accuracy: 0.6925
Epoch 103/300
98/98 [==============================] - 2s 18ms/step - loss: 0.7383 - categorical_accuracy: 0.7463 - val_loss: 0.8678 - val_categorical_accuracy: 0.6970
Epoch 104/300
98/98 [==============================] - 2s 18ms/step - loss: 0.7323 - categorical_accuracy: 0.7484 - val_loss: 0.8695 - val_categorical_accuracy: 0.6976
Epoch 

Epoch 148/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5218 - categorical_accuracy: 0.8294 - val_loss: 0.8003 - val_categorical_accuracy: 0.7254
Epoch 149/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5161 - categorical_accuracy: 0.8300 - val_loss: 0.8295 - val_categorical_accuracy: 0.7158
Epoch 150/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5131 - categorical_accuracy: 0.8312 - val_loss: 0.8165 - val_categorical_accuracy: 0.7181
Epoch 151/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5071 - categorical_accuracy: 0.8333 - val_loss: 0.8063 - val_categorical_accuracy: 0.7228
Epoch 152/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5045 - categorical_accuracy: 0.8335 - val_loss: 0.8070 - val_categorical_accuracy: 0.7248
Epoch 153/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5031 - categorical_accuracy: 0.8338 - val_loss: 0.8107 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3275 - categorical_accuracy: 0.9035 - val_loss: 0.8372 - val_categorical_accuracy: 0.7285
Epoch 198/300
98/98 [==============================] - 2s 19ms/step - loss: 0.3212 - categorical_accuracy: 0.9059 - val_loss: 0.8437 - val_categorical_accuracy: 0.7239
Epoch 199/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3177 - categorical_accuracy: 0.9073 - val_loss: 0.8247 - val_categorical_accuracy: 0.7288
Epoch 200/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3132 - categorical_accuracy: 0.9098 - val_loss: 0.8315 - val_categorical_accuracy: 0.7287
Epoch 201/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3103 - categorical_accuracy: 0.9102 - val_loss: 0.8321 - val_categorical_accuracy: 0.7249
Epoch 202/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3063 - categorical_accuracy: 0.9119 - val_loss: 0.8347 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1702 - categorical_accuracy: 0.9654 - val_loss: 0.9122 - val_categorical_accuracy: 0.7266
Epoch 247/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1672 - categorical_accuracy: 0.9663 - val_loss: 0.9281 - val_categorical_accuracy: 0.7202
Epoch 248/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1642 - categorical_accuracy: 0.9680 - val_loss: 0.9282 - val_categorical_accuracy: 0.7265
Epoch 249/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1624 - categorical_accuracy: 0.9681 - val_loss: 0.9316 - val_categorical_accuracy: 0.7231
Epoch 250/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1605 - categorical_accuracy: 0.9693 - val_loss: 0.9280 - val_categorical_accuracy: 0.7259
Epoch 251/300
98/98 [==============================] - 2s 18ms/step - loss: 0.1556 - categorical_accuracy: 0.9711 - val_loss: 0.9568 - val_categorical_accuracy:

...

98/98 [==============================] - 2s 18ms/step - loss: 0.9067 - categorical_accuracy: 0.6854 - val_loss: 0.9862 - val_categorical_accuracy: 0.6511
Epoch 99/300
98/98 [==============================] - 2s 18ms/step - loss: 0.9008 - categorical_accuracy: 0.6860 - val_loss: 0.9743 - val_categorical_accuracy: 0.6603
Epoch 100/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8945 - categorical_accuracy: 0.6896 - val_loss: 0.9700 - val_categorical_accuracy: 0.6601
Epoch 101/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8937 - categorical_accuracy: 0.6888 - val_loss: 0.9633 - val_categorical_accuracy: 0.6654
Epoch 102/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8893 - categorical_accuracy: 0.6903 - val_loss: 0.9633 - val_categorical_accuracy: 0.6603
Epoch 103/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8802 - categorical_accuracy: 0.6944 - val_loss: 0.9651 - val_categorical_accuracy: 0.6603
Epoch 1

Epoch 147/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6941 - categorical_accuracy: 0.7621 - val_loss: 0.8471 - val_categorical_accuracy: 0.7030
Epoch 148/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6891 - categorical_accuracy: 0.7651 - val_loss: 0.8592 - val_categorical_accuracy: 0.7033
Epoch 149/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6838 - categorical_accuracy: 0.7674 - val_loss: 0.8491 - val_categorical_accuracy: 0.7018
Epoch 150/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6800 - categorical_accuracy: 0.7691 - val_loss: 0.8457 - val_categorical_accuracy: 0.7058
Epoch 151/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6794 - categorical_accuracy: 0.7684 - val_loss: 0.8460 - val_categorical_accuracy: 0.7058
Epoch 152/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6702 - categorical_accuracy: 0.7729 - val_loss: 0.8373 - val_categorical_accuracy:

Epoch 196/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5207 - categorical_accuracy: 0.8283 - val_loss: 0.8005 - val_categorical_accuracy: 0.7260
Epoch 197/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5223 - categorical_accuracy: 0.8258 - val_loss: 0.7990 - val_categorical_accuracy: 0.7287
Epoch 198/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5150 - categorical_accuracy: 0.8300 - val_loss: 0.7993 - val_categorical_accuracy: 0.7272
Epoch 199/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5127 - categorical_accuracy: 0.8303 - val_loss: 0.7956 - val_categorical_accuracy: 0.7269
Epoch 200/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5088 - categorical_accuracy: 0.8326 - val_loss: 0.7957 - val_categorical_accuracy: 0.7296
Epoch 201/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5053 - categorical_accuracy: 0.8341 - val_loss: 0.7977 - val_categorical_accuracy:

Epoch 245/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3737 - categorical_accuracy: 0.8849 - val_loss: 0.7979 - val_categorical_accuracy: 0.7352
Epoch 246/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3713 - categorical_accuracy: 0.8850 - val_loss: 0.7982 - val_categorical_accuracy: 0.7345
Epoch 247/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3704 - categorical_accuracy: 0.8858 - val_loss: 0.8122 - val_categorical_accuracy: 0.7346
Epoch 248/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3678 - categorical_accuracy: 0.8865 - val_loss: 0.8027 - val_categorical_accuracy: 0.7365
Epoch 249/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3628 - categorical_accuracy: 0.8895 - val_loss: 0.8083 - val_categorical_accuracy: 0.7345
Epoch 250/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3616 - categorical_accuracy: 0.8883 - val_loss: 0.8015 - val_categorical_accuracy:

Epoch 294/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2479 - categorical_accuracy: 0.9350 - val_loss: 0.8490 - val_categorical_accuracy: 0.7324
Epoch 295/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2459 - categorical_accuracy: 0.9354 - val_loss: 0.8407 - val_categorical_accuracy: 0.7348
Epoch 296/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2435 - categorical_accuracy: 0.9361 - val_loss: 0.8427 - val_categorical_accuracy: 0.7321
Epoch 297/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2436 - categorical_accuracy: 0.9364 - val_loss: 0.8482 - val_categorical_accuracy: 0.7360
Epoch 298/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2402 - categorical_accuracy: 0.9384 - val_loss: 0.8584 - val_categorical_accuracy: 0.7292
Epoch 299/300
98/98 [==============================] - 2s 18ms/step - loss: 0.2392 - categorical_accuracy: 0.9380 - val_loss: 0.8452 - val_categorical_accuracy:

In [11]:
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid

### Optimizers 
Adam RMSprop SGD

In [12]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = [SGD,Adam,RMSprop]
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



for op in OPTIMIZER: 
    if op == SGD:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(learning_rates, momentum=momentums),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL) 

    elif op == Adam:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=Adam(lr),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)
    elif op == RMSprop:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=RMSprop(learning_rates,momentum=momentums),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)

Epoch 1/300
98/98 [==============================] - 3s 23ms/step - loss: 2.2857 - categorical_accuracy: 0.1441 - val_loss: 2.2670 - val_categorical_accuracy: 0.1620
Epoch 2/300
98/98 [==============================] - 2s 18ms/step - loss: 2.2458 - categorical_accuracy: 0.1895 - val_loss: 2.2162 - val_categorical_accuracy: 0.2246
Epoch 3/300
98/98 [==============================] - 2s 18ms/step - loss: 2.1753 - categorical_accuracy: 0.2336 - val_loss: 2.1230 - val_categorical_accuracy: 0.2528
Epoch 4/300
98/98 [==============================] - 2s 18ms/step - loss: 2.0832 - categorical_accuracy: 0.2570 - val_loss: 2.0450 - val_categorical_accuracy: 0.2669
Epoch 5/300
98/98 [==============================] - 2s 18ms/step - loss: 2.0247 - categorical_accuracy: 0.2752 - val_loss: 1.9978 - val_categorical_accuracy: 0.2892
Epoch 6/300
98/98 [==============================] - 2s 18ms/step - loss: 1.9854 - categorical_accuracy: 0.2905 - val_loss: 1.9621 - val_categorical_accuracy: 0.3091
Epoc

98/98 [==============================] - 2s 18ms/step - loss: 0.8960 - categorical_accuracy: 0.6870 - val_loss: 0.9706 - val_categorical_accuracy: 0.6559
Epoch 100/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8922 - categorical_accuracy: 0.6891 - val_loss: 0.9683 - val_categorical_accuracy: 0.6597
Epoch 101/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8907 - categorical_accuracy: 0.6894 - val_loss: 0.9753 - val_categorical_accuracy: 0.6603
Epoch 102/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8850 - categorical_accuracy: 0.6917 - val_loss: 0.9577 - val_categorical_accuracy: 0.6613
Epoch 103/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8763 - categorical_accuracy: 0.6951 - val_loss: 0.9545 - val_categorical_accuracy: 0.6646
Epoch 104/300
98/98 [==============================] - 2s 18ms/step - loss: 0.8725 - categorical_accuracy: 0.6960 - val_loss: 0.9496 - val_categorical_accuracy: 0.6655
Epoch 

Epoch 148/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6789 - categorical_accuracy: 0.7686 - val_loss: 0.8483 - val_categorical_accuracy: 0.7026
Epoch 149/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6739 - categorical_accuracy: 0.7706 - val_loss: 0.8559 - val_categorical_accuracy: 0.7042
Epoch 150/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6699 - categorical_accuracy: 0.7719 - val_loss: 0.8418 - val_categorical_accuracy: 0.7064
Epoch 151/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6665 - categorical_accuracy: 0.7726 - val_loss: 0.8412 - val_categorical_accuracy: 0.7082
Epoch 152/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6617 - categorical_accuracy: 0.7746 - val_loss: 0.8366 - val_categorical_accuracy: 0.7112
Epoch 153/300
98/98 [==============================] - 2s 18ms/step - loss: 0.6590 - categorical_accuracy: 0.7777 - val_loss: 0.8443 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5056 - categorical_accuracy: 0.8330 - val_loss: 0.7959 - val_categorical_accuracy: 0.7294
Epoch 198/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5007 - categorical_accuracy: 0.8358 - val_loss: 0.8045 - val_categorical_accuracy: 0.7233
Epoch 199/300
98/98 [==============================] - 2s 18ms/step - loss: 0.5030 - categorical_accuracy: 0.8331 - val_loss: 0.8076 - val_categorical_accuracy: 0.7267
Epoch 200/300
98/98 [==============================] - 2s 18ms/step - loss: 0.4975 - categorical_accuracy: 0.8370 - val_loss: 0.8037 - val_categorical_accuracy: 0.7254
Epoch 201/300
98/98 [==============================] - 2s 18ms/step - loss: 0.4907 - categorical_accuracy: 0.8413 - val_loss: 0.8000 - val_categorical_accuracy: 0.7306
Epoch 202/300
98/98 [==============================] - 2s 18ms/step - loss: 0.4866 - categorical_accuracy: 0.8416 - val_loss: 0.7995 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3532 - categorical_accuracy: 0.8935 - val_loss: 0.8064 - val_categorical_accuracy: 0.7328
Epoch 247/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3495 - categorical_accuracy: 0.8944 - val_loss: 0.8093 - val_categorical_accuracy: 0.7350
Epoch 248/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3515 - categorical_accuracy: 0.8939 - val_loss: 0.8065 - val_categorical_accuracy: 0.7345
Epoch 249/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3445 - categorical_accuracy: 0.8960 - val_loss: 0.8034 - val_categorical_accuracy: 0.7326
Epoch 250/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3410 - categorical_accuracy: 0.8979 - val_loss: 0.8059 - val_categorical_accuracy: 0.7343
Epoch 251/300
98/98 [==============================] - 2s 18ms/step - loss: 0.3374 - categorical_accuracy: 0.8993 - val_loss: 0.8040 - val_categorical_accuracy:

...

98/98 [==============================] - 2s 18ms/step - loss: 2.3357 - categorical_accuracy: 0.0999 - val_loss: 2.3322 - val_categorical_accuracy: 0.1000
Epoch 33/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3344 - categorical_accuracy: 0.1001 - val_loss: 2.3281 - val_categorical_accuracy: 0.1000
Epoch 34/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3321 - categorical_accuracy: 0.1003 - val_loss: 2.3352 - val_categorical_accuracy: 0.1000
Epoch 35/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3386 - categorical_accuracy: 0.1012 - val_loss: 2.3339 - val_categorical_accuracy: 0.1000
Epoch 36/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3295 - categorical_accuracy: 0.0982 - val_loss: 2.3203 - val_categorical_accuracy: 0.1000
Epoch 37/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3327 - categorical_accuracy: 0.0990 - val_loss: 2.3828 - val_categorical_accuracy: 0.1000
Epoch 38/30

Epoch 130/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3335 - categorical_accuracy: 0.1006 - val_loss: 2.3382 - val_categorical_accuracy: 0.1000
Epoch 131/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3277 - categorical_accuracy: 0.0987 - val_loss: 2.3263 - val_categorical_accuracy: 0.1000
Epoch 132/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3313 - categorical_accuracy: 0.0991 - val_loss: 2.3288 - val_categorical_accuracy: 0.1000
Epoch 133/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3354 - categorical_accuracy: 0.1002 - val_loss: 2.3421 - val_categorical_accuracy: 0.1000
Epoch 134/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3315 - categorical_accuracy: 0.1001 - val_loss: 2.3207 - val_categorical_accuracy: 0.1000
Epoch 135/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3292 - categorical_accuracy: 0.1030 - val_loss: 2.3392 - val_categorical_accuracy:

Epoch 179/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3355 - categorical_accuracy: 0.1022 - val_loss: 2.3317 - val_categorical_accuracy: 0.1000
Epoch 180/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3325 - categorical_accuracy: 0.1002 - val_loss: 2.3322 - val_categorical_accuracy: 0.1000
Epoch 181/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3331 - categorical_accuracy: 0.1021 - val_loss: 2.3405 - val_categorical_accuracy: 0.1000
Epoch 182/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3364 - categorical_accuracy: 0.0985 - val_loss: 2.3259 - val_categorical_accuracy: 0.1000
Epoch 183/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3321 - categorical_accuracy: 0.0999 - val_loss: 2.3607 - val_categorical_accuracy: 0.1000
Epoch 184/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3382 - categorical_accuracy: 0.1019 - val_loss: 2.3129 - val_categorical_accuracy:

Epoch 228/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3296 - categorical_accuracy: 0.0981 - val_loss: 2.3269 - val_categorical_accuracy: 0.1000
Epoch 229/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3313 - categorical_accuracy: 0.1012 - val_loss: 2.3260 - val_categorical_accuracy: 0.1000
Epoch 230/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3338 - categorical_accuracy: 0.0980 - val_loss: 2.3196 - val_categorical_accuracy: 0.1000
Epoch 231/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3316 - categorical_accuracy: 0.0994 - val_loss: 2.3301 - val_categorical_accuracy: 0.1000
Epoch 232/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3318 - categorical_accuracy: 0.0992 - val_loss: 2.3232 - val_categorical_accuracy: 0.1000
Epoch 233/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3350 - categorical_accuracy: 0.1011 - val_loss: 2.3239 - val_categorical_accuracy:

Epoch 277/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3355 - categorical_accuracy: 0.0998 - val_loss: 2.3225 - val_categorical_accuracy: 0.1000
Epoch 278/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3277 - categorical_accuracy: 0.1001 - val_loss: 2.3329 - val_categorical_accuracy: 0.1000
Epoch 279/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3362 - categorical_accuracy: 0.0994 - val_loss: 2.3201 - val_categorical_accuracy: 0.1000
Epoch 280/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3274 - categorical_accuracy: 0.0991 - val_loss: 2.3492 - val_categorical_accuracy: 0.1000
Epoch 281/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3300 - categorical_accuracy: 0.0999 - val_loss: 2.3187 - val_categorical_accuracy: 0.1000
Epoch 282/300
98/98 [==============================] - 2s 18ms/step - loss: 2.3366 - categorical_accuracy: 0.1019 - val_loss: 2.3427 - val_categorical_accuracy:

### Fonction d'activation 
Tanh sigmoid relu

NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

In [13]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=sigmoid)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

Epoch 1/300
98/98 [==============================] - 2s 17ms/step - loss: 2.3197 - categorical_accuracy: 0.0993 - val_loss: 2.3052 - val_categorical_accuracy: 0.1000
Epoch 2/300
98/98 [==============================] - 1s 15ms/step - loss: 2.3043 - categorical_accuracy: 0.1007 - val_loss: 2.3040 - val_categorical_accuracy: 0.1000
Epoch 3/300
98/98 [==============================] - 1s 15ms/step - loss: 2.3043 - categorical_accuracy: 0.0991 - val_loss: 2.3054 - val_categorical_accuracy: 0.1000
Epoch 4/300
98/98 [==============================] - 1s 15ms/step - loss: 2.3050 - categorical_accuracy: 0.1008 - val_loss: 2.3049 - val_categorical_accuracy: 0.1000
Epoch 5/300
98/98 [==============================] - 1s 15ms/step - loss: 2.3044 - categorical_accuracy: 0.0974 - val_loss: 2.3038 - val_categorical_accuracy: 0.1000
Epoch 6/300
98/98 [==============================] - 1s 15ms/step - loss: 2.3047 - categorical_accuracy: 0.1000 - val_loss: 2.3064 - val_categorical_accuracy: 0.1000
Epoc

98/98 [==============================] - 1s 15ms/step - loss: 2.2466 - categorical_accuracy: 0.1866 - val_loss: 2.2416 - val_categorical_accuracy: 0.2102
Epoch 100/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2401 - categorical_accuracy: 0.1963 - val_loss: 2.2342 - val_categorical_accuracy: 0.1803
Epoch 101/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2321 - categorical_accuracy: 0.2043 - val_loss: 2.2249 - val_categorical_accuracy: 0.2063
Epoch 102/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2240 - categorical_accuracy: 0.2057 - val_loss: 2.2158 - val_categorical_accuracy: 0.2222
Epoch 103/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2153 - categorical_accuracy: 0.2084 - val_loss: 2.2058 - val_categorical_accuracy: 0.2212
Epoch 104/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2051 - categorical_accuracy: 0.2133 - val_loss: 2.1967 - val_categorical_accuracy: 0.2069
Epoch 

Epoch 148/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0273 - categorical_accuracy: 0.2690 - val_loss: 2.0141 - val_categorical_accuracy: 0.2791
Epoch 149/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0263 - categorical_accuracy: 0.2703 - val_loss: 2.0131 - val_categorical_accuracy: 0.2796
Epoch 150/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0240 - categorical_accuracy: 0.2708 - val_loss: 2.0123 - val_categorical_accuracy: 0.2811
Epoch 151/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0232 - categorical_accuracy: 0.2714 - val_loss: 2.0118 - val_categorical_accuracy: 0.2817
Epoch 152/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0223 - categorical_accuracy: 0.2719 - val_loss: 2.0101 - val_categorical_accuracy: 0.2805
Epoch 153/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0212 - categorical_accuracy: 0.2726 - val_loss: 2.0074 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9781 - categorical_accuracy: 0.2905 - val_loss: 1.9659 - val_categorical_accuracy: 0.3006
Epoch 198/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9773 - categorical_accuracy: 0.2918 - val_loss: 1.9642 - val_categorical_accuracy: 0.3086
Epoch 199/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9757 - categorical_accuracy: 0.2929 - val_loss: 1.9642 - val_categorical_accuracy: 0.3096
Epoch 200/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9751 - categorical_accuracy: 0.2929 - val_loss: 1.9615 - val_categorical_accuracy: 0.3081
Epoch 201/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9741 - categorical_accuracy: 0.2913 - val_loss: 1.9623 - val_categorical_accuracy: 0.2981
Epoch 202/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9735 - categorical_accuracy: 0.2924 - val_loss: 1.9603 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9162 - categorical_accuracy: 0.3171 - val_loss: 1.9060 - val_categorical_accuracy: 0.3257
Epoch 247/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9150 - categorical_accuracy: 0.3171 - val_loss: 1.9051 - val_categorical_accuracy: 0.3252
Epoch 248/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9132 - categorical_accuracy: 0.3178 - val_loss: 1.9021 - val_categorical_accuracy: 0.3265
Epoch 249/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9120 - categorical_accuracy: 0.3170 - val_loss: 1.9001 - val_categorical_accuracy: 0.3289
Epoch 250/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9103 - categorical_accuracy: 0.3189 - val_loss: 1.8998 - val_categorical_accuracy: 0.3260
Epoch 251/300
98/98 [==============================] - 1s 15ms/step - loss: 1.9080 - categorical_accuracy: 0.3189 - val_loss: 1.8975 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8220 - categorical_accuracy: 0.3511 - val_loss: 1.8146 - val_categorical_accuracy: 0.3579
Epoch 296/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8200 - categorical_accuracy: 0.3516 - val_loss: 1.8125 - val_categorical_accuracy: 0.3576
Epoch 297/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8172 - categorical_accuracy: 0.3540 - val_loss: 1.8112 - val_categorical_accuracy: 0.3582
Epoch 298/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8159 - categorical_accuracy: 0.3538 - val_loss: 1.8095 - val_categorical_accuracy: 0.3642
Epoch 299/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8120 - categorical_accuracy: 0.3551 - val_loss: 1.8058 - val_categorical_accuracy: 0.3622
Epoch 300/300
98/98 [==============================] - 1s 15ms/step - loss: 1.8103 - categorical_accuracy: 0.3560 - val_loss: 1.8033 - val_categorical_accuracy:

In [14]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=relu)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

Epoch 1/300
98/98 [==============================] - 2s 21ms/step - loss: 2.2994 - categorical_accuracy: 0.1180 - val_loss: 2.2961 - val_categorical_accuracy: 0.1260
Epoch 2/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2932 - categorical_accuracy: 0.1361 - val_loss: 2.2897 - val_categorical_accuracy: 0.1381
Epoch 3/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2854 - categorical_accuracy: 0.1556 - val_loss: 2.2799 - val_categorical_accuracy: 0.1496
Epoch 4/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2722 - categorical_accuracy: 0.1665 - val_loss: 2.2615 - val_categorical_accuracy: 0.1761
Epoch 5/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2439 - categorical_accuracy: 0.1872 - val_loss: 2.2185 - val_categorical_accuracy: 0.1959
Epoch 6/300
98/98 [==============================] - 1s 15ms/step - loss: 2.1746 - categorical_accuracy: 0.2311 - val_loss: 2.1149 - val_categorical_accuracy: 0.2610
Epoc

98/98 [==============================] - 1s 15ms/step - loss: 0.8522 - categorical_accuracy: 0.7043 - val_loss: 1.0163 - val_categorical_accuracy: 0.6449
Epoch 100/300
98/98 [==============================] - 1s 15ms/step - loss: 0.8544 - categorical_accuracy: 0.7038 - val_loss: 1.0024 - val_categorical_accuracy: 0.6508
Epoch 101/300
98/98 [==============================] - 1s 15ms/step - loss: 0.8417 - categorical_accuracy: 0.7078 - val_loss: 1.0026 - val_categorical_accuracy: 0.6540
Epoch 102/300
98/98 [==============================] - 1s 15ms/step - loss: 0.8306 - categorical_accuracy: 0.7123 - val_loss: 1.0202 - val_categorical_accuracy: 0.6465
Epoch 103/300
98/98 [==============================] - 1s 15ms/step - loss: 0.8310 - categorical_accuracy: 0.7108 - val_loss: 0.9871 - val_categorical_accuracy: 0.6599
Epoch 104/300
98/98 [==============================] - 1s 15ms/step - loss: 0.8346 - categorical_accuracy: 0.7091 - val_loss: 0.9895 - val_categorical_accuracy: 0.6555
Epoch 

Epoch 148/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5743 - categorical_accuracy: 0.8045 - val_loss: 1.0093 - val_categorical_accuracy: 0.6613
Epoch 149/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5797 - categorical_accuracy: 0.8015 - val_loss: 0.9927 - val_categorical_accuracy: 0.6760
Epoch 150/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5659 - categorical_accuracy: 0.8065 - val_loss: 0.9846 - val_categorical_accuracy: 0.6765
Epoch 151/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5624 - categorical_accuracy: 0.8074 - val_loss: 0.9794 - val_categorical_accuracy: 0.6789
Epoch 152/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5547 - categorical_accuracy: 0.8107 - val_loss: 1.0115 - val_categorical_accuracy: 0.6674
Epoch 153/300
98/98 [==============================] - 1s 15ms/step - loss: 0.5644 - categorical_accuracy: 0.8072 - val_loss: 1.0064 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 1s 15ms/step - loss: 0.3263 - categorical_accuracy: 0.8923 - val_loss: 1.1832 - val_categorical_accuracy: 0.6744
Epoch 198/300
98/98 [==============================] - 1s 15ms/step - loss: 0.3135 - categorical_accuracy: 0.8970 - val_loss: 1.2101 - val_categorical_accuracy: 0.6687
Epoch 199/300
98/98 [==============================] - 1s 15ms/step - loss: 0.3041 - categorical_accuracy: 0.9000 - val_loss: 1.2034 - val_categorical_accuracy: 0.6700
Epoch 200/300
98/98 [==============================] - 1s 15ms/step - loss: 0.3103 - categorical_accuracy: 0.8980 - val_loss: 1.2530 - val_categorical_accuracy: 0.6605
Epoch 201/300
98/98 [==============================] - 1s 15ms/step - loss: 0.3073 - categorical_accuracy: 0.8970 - val_loss: 1.2004 - val_categorical_accuracy: 0.6735
Epoch 202/300
98/98 [==============================] - 1s 15ms/step - loss: 0.2987 - categorical_accuracy: 0.9017 - val_loss: 1.2802 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 1s 15ms/step - loss: 0.1110 - categorical_accuracy: 0.9707 - val_loss: 1.6847 - val_categorical_accuracy: 0.6632
Epoch 247/300
98/98 [==============================] - 1s 15ms/step - loss: 0.1112 - categorical_accuracy: 0.9706 - val_loss: 1.6868 - val_categorical_accuracy: 0.6698
Epoch 248/300
98/98 [==============================] - 1s 15ms/step - loss: 0.1176 - categorical_accuracy: 0.9671 - val_loss: 1.7615 - val_categorical_accuracy: 0.6599
Epoch 249/300
98/98 [==============================] - 1s 15ms/step - loss: 0.1170 - categorical_accuracy: 0.9669 - val_loss: 1.7439 - val_categorical_accuracy: 0.6618
Epoch 250/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0976 - categorical_accuracy: 0.9760 - val_loss: 1.7266 - val_categorical_accuracy: 0.6664
Epoch 251/300
98/98 [==============================] - 1s 15ms/step - loss: 0.1111 - categorical_accuracy: 0.9685 - val_loss: 1.8209 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0195 - categorical_accuracy: 0.9992 - val_loss: 2.2850 - val_categorical_accuracy: 0.6637
Epoch 296/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0156 - categorical_accuracy: 0.9997 - val_loss: 2.3051 - val_categorical_accuracy: 0.6625
Epoch 297/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0161 - categorical_accuracy: 0.9995 - val_loss: 2.3119 - val_categorical_accuracy: 0.6626
Epoch 298/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0156 - categorical_accuracy: 0.9995 - val_loss: 2.3300 - val_categorical_accuracy: 0.6622
Epoch 299/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0154 - categorical_accuracy: 0.9996 - val_loss: 2.3356 - val_categorical_accuracy: 0.6637
Epoch 300/300
98/98 [==============================] - 1s 15ms/step - loss: 0.0145 - categorical_accuracy: 0.9996 - val_loss: 2.3458 - val_categorical_accuracy:

### Structure Pooling

In [15]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = tf.keras.layers.AveragePooling2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_AVGPOOLING_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/300
98/98 [==============================] - 2s 18ms/step - loss: 2.2992 - categorical_accuracy: 0.1169 - val_loss: 2.2895 - val_categorical_accuracy: 0.1202
Epoch 2/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2810 - categorical_accuracy: 0.1311 - val_loss: 2.2704 - val_categorical_accuracy: 0.1561
Epoch 3/300
98/98 [==============================] - 2s 15ms/step - loss: 2.2570 - categorical_accuracy: 0.1770 - val_loss: 2.2389 - val_categorical_accuracy: 0.2094
Epoch 4/300
98/98 [==============================] - 1s 15ms/step - loss: 2.2132 - categorical_accuracy: 0.2187 - val_loss: 2.1773 - val_categorical_accuracy: 0.2279
Epoch 5/300
98/98 [==============================] - 1s 15ms/step - loss: 2.1386 - categorical_accuracy: 0.2283 - val_loss: 2.0957 - val_categorical_accuracy: 0.2414
Epoch 6/300
98/98 [==============================] - 1s 15ms/step - loss: 2.0784 - categorical_accuracy: 0.2429 - val_loss: 2.0559 - val_categorical_accuracy: 0.2617
Epoc

98/98 [==============================] - 1s 15ms/step - loss: 1.5342 - categorical_accuracy: 0.4584 - val_loss: 1.5255 - val_categorical_accuracy: 0.4536
Epoch 100/300
98/98 [==============================] - 2s 15ms/step - loss: 1.5314 - categorical_accuracy: 0.4582 - val_loss: 1.5214 - val_categorical_accuracy: 0.4577
Epoch 101/300
98/98 [==============================] - 1s 15ms/step - loss: 1.5261 - categorical_accuracy: 0.4612 - val_loss: 1.5178 - val_categorical_accuracy: 0.4554
Epoch 102/300
98/98 [==============================] - 1s 15ms/step - loss: 1.5226 - categorical_accuracy: 0.4613 - val_loss: 1.5126 - val_categorical_accuracy: 0.4590
Epoch 103/300
98/98 [==============================] - 1s 15ms/step - loss: 1.5183 - categorical_accuracy: 0.4625 - val_loss: 1.5112 - val_categorical_accuracy: 0.4566
Epoch 104/300
98/98 [==============================] - 2s 15ms/step - loss: 1.5140 - categorical_accuracy: 0.4634 - val_loss: 1.5049 - val_categorical_accuracy: 0.4642
Epoch 

Epoch 148/300
98/98 [==============================] - 2s 15ms/step - loss: 1.3721 - categorical_accuracy: 0.5103 - val_loss: 1.3685 - val_categorical_accuracy: 0.5094
Epoch 149/300
98/98 [==============================] - 1s 15ms/step - loss: 1.3704 - categorical_accuracy: 0.5106 - val_loss: 1.3753 - val_categorical_accuracy: 0.5098
Epoch 150/300
98/98 [==============================] - 1s 15ms/step - loss: 1.3676 - categorical_accuracy: 0.5134 - val_loss: 1.3655 - val_categorical_accuracy: 0.5135
Epoch 151/300
98/98 [==============================] - 2s 15ms/step - loss: 1.3658 - categorical_accuracy: 0.5136 - val_loss: 1.3636 - val_categorical_accuracy: 0.5145
Epoch 152/300
98/98 [==============================] - 2s 15ms/step - loss: 1.3625 - categorical_accuracy: 0.5147 - val_loss: 1.3601 - val_categorical_accuracy: 0.5123
Epoch 153/300
98/98 [==============================] - 1s 15ms/step - loss: 1.3616 - categorical_accuracy: 0.5151 - val_loss: 1.3611 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2822 - categorical_accuracy: 0.5456 - val_loss: 1.2899 - val_categorical_accuracy: 0.5407
Epoch 198/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2807 - categorical_accuracy: 0.5469 - val_loss: 1.2900 - val_categorical_accuracy: 0.5376
Epoch 199/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2802 - categorical_accuracy: 0.5465 - val_loss: 1.2864 - val_categorical_accuracy: 0.5391
Epoch 200/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2784 - categorical_accuracy: 0.5480 - val_loss: 1.2859 - val_categorical_accuracy: 0.5423
Epoch 201/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2778 - categorical_accuracy: 0.5473 - val_loss: 1.2849 - val_categorical_accuracy: 0.5409
Epoch 202/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2756 - categorical_accuracy: 0.5492 - val_loss: 1.2831 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2128 - categorical_accuracy: 0.5739 - val_loss: 1.2330 - val_categorical_accuracy: 0.5652
Epoch 247/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2110 - categorical_accuracy: 0.5762 - val_loss: 1.2328 - val_categorical_accuracy: 0.5654
Epoch 248/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2100 - categorical_accuracy: 0.5756 - val_loss: 1.2343 - val_categorical_accuracy: 0.5639
Epoch 249/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2090 - categorical_accuracy: 0.5753 - val_loss: 1.2317 - val_categorical_accuracy: 0.5668
Epoch 250/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2081 - categorical_accuracy: 0.5766 - val_loss: 1.2304 - val_categorical_accuracy: 0.5652
Epoch 251/300
98/98 [==============================] - 1s 15ms/step - loss: 1.2057 - categorical_accuracy: 0.5776 - val_loss: 1.2291 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1509 - categorical_accuracy: 0.5972 - val_loss: 1.1893 - val_categorical_accuracy: 0.5804
Epoch 296/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1502 - categorical_accuracy: 0.5975 - val_loss: 1.1844 - val_categorical_accuracy: 0.5839
Epoch 297/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1502 - categorical_accuracy: 0.5977 - val_loss: 1.1859 - val_categorical_accuracy: 0.5834
Epoch 298/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1479 - categorical_accuracy: 0.5988 - val_loss: 1.1834 - val_categorical_accuracy: 0.5833
Epoch 299/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1455 - categorical_accuracy: 0.6001 - val_loss: 1.1846 - val_categorical_accuracy: 0.5851
Epoch 300/300
98/98 [==============================] - 1s 15ms/step - loss: 1.1464 - categorical_accuracy: 0.5996 - val_loss: 1.1804 - val_categorical_accuracy:

In [10]:
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


convnet = Sequential() 
convnet.add(Conv2D(128,input_shape=(32, 32, 3), kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(Conv2D(64,kernel_size=(3, 3), padding="same", activation=tanh))  
convnet.add(Conv2D(32,kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(MaxPool2D())  
convnet.add(Flatten())
convnet.add(Dense(NUM_CLASSES, activation=softmax))

convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_MAXPOOLING_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers__af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet__sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers__af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/300
98/98 [==============================] - 10s 68ms/step - loss: 2.2270 - categorical_accuracy: 0.1936 - val_loss: 2.1022 - val_categorical_accuracy: 0.2775
Epoch 2/300
98/98 [==============================] - 5s 55ms/step - loss: 1.9879 - categorical_accuracy: 0.3125 - val_loss: 1.9086 - val_categorical_accuracy: 0.3399
Epoch 3/300
98/98 [==============================] - 5s 55ms/step - loss: 1.8747 - categorical_accuracy: 0.3544 - val_loss: 1.8351 - val_categorical_accuracy: 0.3693
Epoch 4/300
98/98 [==============================] - 5s 55ms/step - loss: 1.8057 - categorical_accuracy: 0.3759 - val_loss: 1.7666 - val_categorical_accuracy: 0.3887
Epoch 5/300
98/98 [==============================] - 6s 56ms/step - loss: 1.7513 - categorical_accuracy: 0.3932 - val_loss: 1.7233 - val_categorical_accuracy: 0.3994
Epoch 6/300
98/98 [==============================] - 5s 56ms/step - loss: 1.7107 - categorical_accuracy: 0.4051 - val_loss: 1.6884 - val_categorical_accuracy: 0.4139
Epo

98/98 [==============================] - 5s 55ms/step - loss: 0.7635 - categorical_accuracy: 0.7424 - val_loss: 0.9928 - val_categorical_accuracy: 0.6555
Epoch 100/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7590 - categorical_accuracy: 0.7438 - val_loss: 0.9896 - val_categorical_accuracy: 0.6566
Epoch 101/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7538 - categorical_accuracy: 0.7477 - val_loss: 0.9865 - val_categorical_accuracy: 0.6566
Epoch 102/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7515 - categorical_accuracy: 0.7464 - val_loss: 0.9881 - val_categorical_accuracy: 0.6554
Epoch 103/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7451 - categorical_accuracy: 0.7489 - val_loss: 0.9896 - val_categorical_accuracy: 0.6543
Epoch 104/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7439 - categorical_accuracy: 0.7498 - val_loss: 0.9866 - val_categorical_accuracy: 0.6573
Epoch 

Epoch 196/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5090 - categorical_accuracy: 0.8400 - val_loss: 1.0071 - val_categorical_accuracy: 0.6638
Epoch 197/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5069 - categorical_accuracy: 0.8399 - val_loss: 1.0097 - val_categorical_accuracy: 0.6649
Epoch 198/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5048 - categorical_accuracy: 0.8415 - val_loss: 1.0099 - val_categorical_accuracy: 0.6636
Epoch 199/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5034 - categorical_accuracy: 0.8421 - val_loss: 1.0101 - val_categorical_accuracy: 0.6613
Epoch 200/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5013 - categorical_accuracy: 0.8419 - val_loss: 1.0105 - val_categorical_accuracy: 0.6592
Epoch 201/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4991 - categorical_accuracy: 0.8435 - val_loss: 1.0131 - val_categorical_accuracy:

Epoch 245/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4294 - categorical_accuracy: 0.8714 - val_loss: 1.0481 - val_categorical_accuracy: 0.6579
Epoch 246/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4273 - categorical_accuracy: 0.8717 - val_loss: 1.0539 - val_categorical_accuracy: 0.6576
Epoch 247/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4268 - categorical_accuracy: 0.8722 - val_loss: 1.0585 - val_categorical_accuracy: 0.6585
Epoch 248/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4250 - categorical_accuracy: 0.8736 - val_loss: 1.0554 - val_categorical_accuracy: 0.6571
Epoch 249/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4229 - categorical_accuracy: 0.8739 - val_loss: 1.0538 - val_categorical_accuracy: 0.6559
Epoch 250/300
98/98 [==============================] - 5s 56ms/step - loss: 0.4219 - categorical_accuracy: 0.8748 - val_loss: 1.0533 - val_categorical_accuracy:

Epoch 294/300
98/98 [==============================] - 6s 56ms/step - loss: 0.3672 - categorical_accuracy: 0.8974 - val_loss: 1.0986 - val_categorical_accuracy: 0.6508
Epoch 295/300
98/98 [==============================] - 6s 56ms/step - loss: 0.3659 - categorical_accuracy: 0.8973 - val_loss: 1.1064 - val_categorical_accuracy: 0.6491
Epoch 296/300
98/98 [==============================] - 6s 56ms/step - loss: 0.3647 - categorical_accuracy: 0.8988 - val_loss: 1.1046 - val_categorical_accuracy: 0.6511
Epoch 297/300
98/98 [==============================] - 6s 56ms/step - loss: 0.3633 - categorical_accuracy: 0.8980 - val_loss: 1.1066 - val_categorical_accuracy: 0.6477
Epoch 298/300
98/98 [==============================] - 6s 56ms/step - loss: 0.3629 - categorical_accuracy: 0.8996 - val_loss: 1.1051 - val_categorical_accuracy: 0.6490
Epoch 299/300
98/98 [==============================] - 5s 56ms/step - loss: 0.3619 - categorical_accuracy: 0.8988 - val_loss: 1.1105 - val_categorical_accuracy:

In [16]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="valid", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_paddingvalid_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_paddingvalid_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/300
98/98 [==============================] - 2s 16ms/step - loss: 2.2967 - categorical_accuracy: 0.1095 - val_loss: 2.2741 - val_categorical_accuracy: 0.1404
Epoch 2/300
98/98 [==============================] - 1s 11ms/step - loss: 2.2539 - categorical_accuracy: 0.1784 - val_loss: 2.2290 - val_categorical_accuracy: 0.2222
Epoch 3/300
98/98 [==============================] - 1s 11ms/step - loss: 2.1992 - categorical_accuracy: 0.2389 - val_loss: 2.1610 - val_categorical_accuracy: 0.2473
Epoch 4/300
98/98 [==============================] - 1s 11ms/step - loss: 2.1210 - categorical_accuracy: 0.2586 - val_loss: 2.0733 - val_categorical_accuracy: 0.2724
Epoch 5/300
98/98 [==============================] - 1s 11ms/step - loss: 2.0435 - categorical_accuracy: 0.2800 - val_loss: 2.0075 - val_categorical_accuracy: 0.2906
Epoch 6/300
98/98 [==============================] - 1s 11ms/step - loss: 1.9927 - categorical_accuracy: 0.2939 - val_loss: 1.9667 - val_categorical_accuracy: 0.3064
Epoc

98/98 [==============================] - 1s 11ms/step - loss: 1.1269 - categorical_accuracy: 0.6113 - val_loss: 1.1634 - val_categorical_accuracy: 0.5950
Epoch 100/300
98/98 [==============================] - 1s 11ms/step - loss: 1.1244 - categorical_accuracy: 0.6138 - val_loss: 1.1639 - val_categorical_accuracy: 0.5939
Epoch 101/300
98/98 [==============================] - 1s 11ms/step - loss: 1.1226 - categorical_accuracy: 0.6127 - val_loss: 1.1654 - val_categorical_accuracy: 0.5918
Epoch 102/300
98/98 [==============================] - 1s 11ms/step - loss: 1.1168 - categorical_accuracy: 0.6158 - val_loss: 1.1540 - val_categorical_accuracy: 0.5977
Epoch 103/300
98/98 [==============================] - 1s 11ms/step - loss: 1.1132 - categorical_accuracy: 0.6169 - val_loss: 1.1539 - val_categorical_accuracy: 0.5971
Epoch 104/300
98/98 [==============================] - 1s 11ms/step - loss: 1.1094 - categorical_accuracy: 0.6179 - val_loss: 1.1504 - val_categorical_accuracy: 0.6006
Epoch 

Epoch 148/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9793 - categorical_accuracy: 0.6649 - val_loss: 1.0411 - val_categorical_accuracy: 0.6379
Epoch 149/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9773 - categorical_accuracy: 0.6669 - val_loss: 1.0401 - val_categorical_accuracy: 0.6375
Epoch 150/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9728 - categorical_accuracy: 0.6669 - val_loss: 1.0388 - val_categorical_accuracy: 0.6387
Epoch 151/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9719 - categorical_accuracy: 0.6677 - val_loss: 1.0381 - val_categorical_accuracy: 0.6383
Epoch 152/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9687 - categorical_accuracy: 0.6688 - val_loss: 1.0354 - val_categorical_accuracy: 0.6406
Epoch 153/300
98/98 [==============================] - 1s 12ms/step - loss: 0.9657 - categorical_accuracy: 0.6691 - val_loss: 1.0313 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8683 - categorical_accuracy: 0.7036 - val_loss: 0.9642 - val_categorical_accuracy: 0.6658
Epoch 198/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8671 - categorical_accuracy: 0.7053 - val_loss: 0.9672 - val_categorical_accuracy: 0.6630
Epoch 199/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8643 - categorical_accuracy: 0.7063 - val_loss: 0.9605 - val_categorical_accuracy: 0.6690
Epoch 200/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8627 - categorical_accuracy: 0.7063 - val_loss: 0.9591 - val_categorical_accuracy: 0.6710
Epoch 201/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8612 - categorical_accuracy: 0.7075 - val_loss: 0.9630 - val_categorical_accuracy: 0.6696
Epoch 202/300
98/98 [==============================] - 1s 12ms/step - loss: 0.8599 - categorical_accuracy: 0.7069 - val_loss: 0.9555 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7806 - categorical_accuracy: 0.7355 - val_loss: 0.9105 - val_categorical_accuracy: 0.6862
Epoch 247/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7803 - categorical_accuracy: 0.7347 - val_loss: 0.9172 - val_categorical_accuracy: 0.6838
Epoch 248/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7785 - categorical_accuracy: 0.7366 - val_loss: 0.9143 - val_categorical_accuracy: 0.6808
Epoch 249/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7791 - categorical_accuracy: 0.7366 - val_loss: 0.9083 - val_categorical_accuracy: 0.6856
Epoch 250/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7766 - categorical_accuracy: 0.7374 - val_loss: 0.9082 - val_categorical_accuracy: 0.6865
Epoch 251/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7734 - categorical_accuracy: 0.7390 - val_loss: 0.9167 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7109 - categorical_accuracy: 0.7623 - val_loss: 0.8784 - val_categorical_accuracy: 0.6966
Epoch 296/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7078 - categorical_accuracy: 0.7634 - val_loss: 0.8765 - val_categorical_accuracy: 0.6984
Epoch 297/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7077 - categorical_accuracy: 0.7634 - val_loss: 0.8793 - val_categorical_accuracy: 0.6980
Epoch 298/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7095 - categorical_accuracy: 0.7632 - val_loss: 0.8816 - val_categorical_accuracy: 0.6966
Epoch 299/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7053 - categorical_accuracy: 0.7638 - val_loss: 0.8724 - val_categorical_accuracy: 0.6990
Epoch 300/300
98/98 [==============================] - 1s 11ms/step - loss: 0.7038 - categorical_accuracy: 0.7645 - val_loss: 0.8762 - val_categorical_accuracy:

In [10]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)




num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL) 

Epoch 1/300
98/98 [==============================] - 5s 25ms/step - loss: 2.2931 - categorical_accuracy: 0.1294 - val_loss: 2.2795 - val_categorical_accuracy: 0.1538
Epoch 2/300
98/98 [==============================] - 2s 18ms/step - loss: 2.2663 - categorical_accuracy: 0.1990 - val_loss: 2.2519 - val_categorical_accuracy: 0.2230
Epoch 3/300
98/98 [==============================] - 2s 17ms/step - loss: 2.2328 - categorical_accuracy: 0.2406 - val_loss: 2.2106 - val_categorical_accuracy: 0.2625
Epoch 4/300
98/98 [==============================] - 2s 18ms/step - loss: 2.1788 - categorical_accuracy: 0.2715 - val_loss: 2.1406 - val_categorical_accuracy: 0.2792
Epoch 5/300
98/98 [==============================] - 2s 18ms/step - loss: 2.0930 - categorical_accuracy: 0.2904 - val_loss: 2.0423 - val_categorical_accuracy: 0.3058
Epoch 6/300
98/98 [==============================] - 2s 18ms/step - loss: 2.0067 - categorical_accuracy: 0.3002 - val_loss: 1.9736 - val_categorical_accuracy: 0.3181
Epoc

98/98 [==============================] - 2s 17ms/step - loss: 0.9072 - categorical_accuracy: 0.6829 - val_loss: 0.9714 - val_categorical_accuracy: 0.6542
Epoch 100/300
98/98 [==============================] - 2s 17ms/step - loss: 0.9033 - categorical_accuracy: 0.6847 - val_loss: 0.9658 - val_categorical_accuracy: 0.6588
Epoch 101/300
98/98 [==============================] - 2s 17ms/step - loss: 0.9005 - categorical_accuracy: 0.6844 - val_loss: 0.9643 - val_categorical_accuracy: 0.6590
Epoch 102/300
98/98 [==============================] - 2s 17ms/step - loss: 0.8944 - categorical_accuracy: 0.6867 - val_loss: 0.9644 - val_categorical_accuracy: 0.6606
Epoch 103/300
98/98 [==============================] - 2s 17ms/step - loss: 0.8856 - categorical_accuracy: 0.6916 - val_loss: 0.9558 - val_categorical_accuracy: 0.6614
Epoch 104/300
98/98 [==============================] - 2s 17ms/step - loss: 0.8820 - categorical_accuracy: 0.6930 - val_loss: 0.9502 - val_categorical_accuracy: 0.6657
Epoch 

Epoch 148/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6868 - categorical_accuracy: 0.7665 - val_loss: 0.8408 - val_categorical_accuracy: 0.7091
Epoch 149/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6856 - categorical_accuracy: 0.7663 - val_loss: 0.8386 - val_categorical_accuracy: 0.7050
Epoch 150/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6803 - categorical_accuracy: 0.7683 - val_loss: 0.8423 - val_categorical_accuracy: 0.7067
Epoch 151/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6762 - categorical_accuracy: 0.7706 - val_loss: 0.8325 - val_categorical_accuracy: 0.7105
Epoch 152/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6701 - categorical_accuracy: 0.7727 - val_loss: 0.8322 - val_categorical_accuracy: 0.7142
Epoch 153/300
98/98 [==============================] - 2s 17ms/step - loss: 0.6664 - categorical_accuracy: 0.7749 - val_loss: 0.8386 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 2s 17ms/step - loss: 0.5196 - categorical_accuracy: 0.8283 - val_loss: 0.7925 - val_categorical_accuracy: 0.7281
Epoch 198/300
98/98 [==============================] - 2s 17ms/step - loss: 0.5159 - categorical_accuracy: 0.8297 - val_loss: 0.7946 - val_categorical_accuracy: 0.7294
Epoch 199/300
98/98 [==============================] - 2s 17ms/step - loss: 0.5108 - categorical_accuracy: 0.8318 - val_loss: 0.7832 - val_categorical_accuracy: 0.7291
Epoch 200/300
98/98 [==============================] - 2s 17ms/step - loss: 0.5082 - categorical_accuracy: 0.8334 - val_loss: 0.7903 - val_categorical_accuracy: 0.7291
Epoch 201/300
98/98 [==============================] - 2s 17ms/step - loss: 0.5043 - categorical_accuracy: 0.8337 - val_loss: 0.7917 - val_categorical_accuracy: 0.7284
Epoch 202/300
98/98 [==============================] - 2s 17ms/step - loss: 0.4986 - categorical_accuracy: 0.8367 - val_loss: 0.7999 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3699 - categorical_accuracy: 0.8849 - val_loss: 0.7862 - val_categorical_accuracy: 0.7358
Epoch 247/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3679 - categorical_accuracy: 0.8865 - val_loss: 0.8157 - val_categorical_accuracy: 0.7282
Epoch 248/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3676 - categorical_accuracy: 0.8863 - val_loss: 0.8003 - val_categorical_accuracy: 0.7321
Epoch 249/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3610 - categorical_accuracy: 0.8887 - val_loss: 0.7953 - val_categorical_accuracy: 0.7348
Epoch 250/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3600 - categorical_accuracy: 0.8892 - val_loss: 0.7980 - val_categorical_accuracy: 0.7322
Epoch 251/300
98/98 [==============================] - 2s 17ms/step - loss: 0.3549 - categorical_accuracy: 0.8924 - val_loss: 0.8078 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2436 - categorical_accuracy: 0.9371 - val_loss: 0.8439 - val_categorical_accuracy: 0.7322
Epoch 296/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2407 - categorical_accuracy: 0.9381 - val_loss: 0.8364 - val_categorical_accuracy: 0.7343
Epoch 297/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2411 - categorical_accuracy: 0.9370 - val_loss: 0.8453 - val_categorical_accuracy: 0.7347
Epoch 298/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2359 - categorical_accuracy: 0.9398 - val_loss: 0.8468 - val_categorical_accuracy: 0.7338
Epoch 299/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2359 - categorical_accuracy: 0.9399 - val_loss: 0.8483 - val_categorical_accuracy: 0.7344
Epoch 300/300
98/98 [==============================] - 2s 17ms/step - loss: 0.2315 - categorical_accuracy: 0.9422 - val_loss: 0.8419 - val_categorical_accuracy:

In [9]:
BATCH_SIZE = 512
EPOCHS = 300
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


convnet = Sequential() 
convnet.add(Conv2D(128,input_shape=(32, 32, 3), kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(Conv2D(64,kernel_size=(3, 3), padding="same", activation=tanh))  
convnet.add(Conv2D(32,kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(tf.keras.layers.AveragePooling2D())  
convnet.add(Flatten())
convnet.add(Dense(NUM_CLASSES, activation=softmax))

convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_AVERAGEPOOLING_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers__af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet__sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_AVERAGEPOOLING_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers__af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/300
98/98 [==============================] - 10s 70ms/step - loss: 2.2300 - categorical_accuracy: 0.1917 - val_loss: 2.1067 - val_categorical_accuracy: 0.2674
Epoch 2/300
98/98 [==============================] - 5s 55ms/step - loss: 2.0041 - categorical_accuracy: 0.2978 - val_loss: 1.9404 - val_categorical_accuracy: 0.3237
Epoch 3/300
98/98 [==============================] - 5s 55ms/step - loss: 1.9152 - categorical_accuracy: 0.3328 - val_loss: 1.8881 - val_categorical_accuracy: 0.3482
Epoch 4/300
98/98 [==============================] - 5s 55ms/step - loss: 1.8647 - categorical_accuracy: 0.3531 - val_loss: 1.8362 - val_categorical_accuracy: 0.3653
Epoch 5/300
98/98 [==============================] - 5s 55ms/step - loss: 1.8222 - categorical_accuracy: 0.3694 - val_loss: 1.7997 - val_categorical_accuracy: 0.3766
Epoch 6/300
98/98 [==============================] - 5s 55ms/step - loss: 1.7890 - categorical_accuracy: 0.3792 - val_loss: 1.7733 - val_categorical_accuracy: 0.3878
Epo

98/98 [==============================] - 5s 55ms/step - loss: 1.0456 - categorical_accuracy: 0.6359 - val_loss: 1.1691 - val_categorical_accuracy: 0.5842
Epoch 100/300
98/98 [==============================] - 5s 55ms/step - loss: 1.0395 - categorical_accuracy: 0.6388 - val_loss: 1.1599 - val_categorical_accuracy: 0.5905
Epoch 101/300
98/98 [==============================] - 5s 55ms/step - loss: 1.0356 - categorical_accuracy: 0.6408 - val_loss: 1.1588 - val_categorical_accuracy: 0.5936
Epoch 102/300
98/98 [==============================] - 5s 55ms/step - loss: 1.0322 - categorical_accuracy: 0.6409 - val_loss: 1.1544 - val_categorical_accuracy: 0.5925
Epoch 103/300
98/98 [==============================] - 5s 56ms/step - loss: 1.0262 - categorical_accuracy: 0.6454 - val_loss: 1.1584 - val_categorical_accuracy: 0.5873
Epoch 104/300
98/98 [==============================] - 5s 56ms/step - loss: 1.0230 - categorical_accuracy: 0.6450 - val_loss: 1.1562 - val_categorical_accuracy: 0.5887
Epoch 

Epoch 148/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8603 - categorical_accuracy: 0.7050 - val_loss: 1.0750 - val_categorical_accuracy: 0.6213
Epoch 149/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8565 - categorical_accuracy: 0.7089 - val_loss: 1.0679 - val_categorical_accuracy: 0.6236
Epoch 150/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8530 - categorical_accuracy: 0.7073 - val_loss: 1.0641 - val_categorical_accuracy: 0.6246
Epoch 151/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8488 - categorical_accuracy: 0.7117 - val_loss: 1.0638 - val_categorical_accuracy: 0.6285
Epoch 152/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8467 - categorical_accuracy: 0.7114 - val_loss: 1.0691 - val_categorical_accuracy: 0.6210
Epoch 153/300
98/98 [==============================] - 5s 55ms/step - loss: 0.8438 - categorical_accuracy: 0.7128 - val_loss: 1.0650 - val_categorical_accuracy:

Epoch 197/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7350 - categorical_accuracy: 0.7531 - val_loss: 1.0343 - val_categorical_accuracy: 0.6412
Epoch 198/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7320 - categorical_accuracy: 0.7550 - val_loss: 1.0302 - val_categorical_accuracy: 0.6436
Epoch 199/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7298 - categorical_accuracy: 0.7555 - val_loss: 1.0329 - val_categorical_accuracy: 0.6392
Epoch 200/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7288 - categorical_accuracy: 0.7551 - val_loss: 1.0360 - val_categorical_accuracy: 0.6371
Epoch 201/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7257 - categorical_accuracy: 0.7573 - val_loss: 1.0305 - val_categorical_accuracy: 0.6431
Epoch 202/300
98/98 [==============================] - 5s 55ms/step - loss: 0.7243 - categorical_accuracy: 0.7577 - val_loss: 1.0288 - val_categorical_accuracy:

Epoch 246/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6444 - categorical_accuracy: 0.7878 - val_loss: 1.0191 - val_categorical_accuracy: 0.6520
Epoch 247/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6425 - categorical_accuracy: 0.7892 - val_loss: 1.0228 - val_categorical_accuracy: 0.6486
Epoch 248/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6427 - categorical_accuracy: 0.7886 - val_loss: 1.0299 - val_categorical_accuracy: 0.6441
Epoch 249/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6402 - categorical_accuracy: 0.7875 - val_loss: 1.0230 - val_categorical_accuracy: 0.6483
Epoch 250/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6376 - categorical_accuracy: 0.7895 - val_loss: 1.0209 - val_categorical_accuracy: 0.6502
Epoch 251/300
98/98 [==============================] - 5s 55ms/step - loss: 0.6372 - categorical_accuracy: 0.7892 - val_loss: 1.0219 - val_categorical_accuracy:

Epoch 295/300
98/98 [==============================] - 5s 55ms/step - loss: 0.5740 - categorical_accuracy: 0.8153 - val_loss: 1.0260 - val_categorical_accuracy: 0.6520
Epoch 296/300
98/98 [==============================] - 6s 57ms/step - loss: 0.5737 - categorical_accuracy: 0.8155 - val_loss: 1.0340 - val_categorical_accuracy: 0.6505
Epoch 297/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5721 - categorical_accuracy: 0.8156 - val_loss: 1.0361 - val_categorical_accuracy: 0.6501
Epoch 298/300
98/98 [==============================] - 5s 55ms/step - loss: 0.5700 - categorical_accuracy: 0.8157 - val_loss: 1.0265 - val_categorical_accuracy: 0.6552
Epoch 299/300
98/98 [==============================] - 5s 55ms/step - loss: 0.5675 - categorical_accuracy: 0.8190 - val_loss: 1.0292 - val_categorical_accuracy: 0.6526
Epoch 300/300
98/98 [==============================] - 5s 56ms/step - loss: 0.5678 - categorical_accuracy: 0.8190 - val_loss: 1.0260 - val_categorical_accuracy: